<center>Национальный исследовательский университет ИТМО<br/>Факультет информационных технологий и программирования<br/>Прикладная математика и информатика</center>

## <center>Математическая статистика</center>
### <center>Отчёт по лабораторной работе №5</center>

<div style="text-align: right"><b>Работу выполнили:</b><br/>Белоцерковченко Андрей M3337<br/>Смирнов Андрей M3337<br/</div>

<center>Санкт-Петербург<br/>2023</center>

In [134]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy.stats import t
from scipy.stats import f
alpha = 0.05

## Вариант 3

### Задача 1
Для начала немного обработаем датасет

In [135]:
df = pd.read_csv('MEN_SHOES.csv')
df = df.dropna(subset=['Current_Price'])
Y = np.array(df['RATING'])
df = df.drop(columns=['Brand_Name', 'Product_details', 'RATING'])
current_price = np.array(df['Current_Price'].str[1:].str.replace(',', '').apply(np.float64))
volume = np.array(df['How_Many_Sold'].str.replace(',', '').apply(np.float64))
X = np.array([current_price, volume, np.ones(len(current_price))]).T

Здесь и далее: $c_0$ - стоимость пары, $c_1$ - объём продаж, $c_2$ - свободный

### Оценки
Посчитаем оценку $C$, для этого есть формула: $$\hat{c} = A^{-1}X^TY, A = X^TX$$

In [136]:
A = np.dot(X.T, X)
invA = np.linalg.inv(A)
C = np.dot(np.dot(invA, X.T), Y)

n = len(current_price)
print("Коэффициенты регрессии:", C[0], C[1], C[2])

Коэффициенты регрессии: 0.0005263263156671585 8.263894533593854e-06 3.3475715019143597


Проверим правильность коэффициентов - обучим модель из библиотеки

In [137]:
regression = LinearRegression().fit(X, Y)
print(regression.coef_[0], regression.coef_[1], regression.intercept_)

0.0005263263156671531 8.263894533593864e-06 3.3475715019143637


Теперь посчитаем оценку остаточной дисперсии, для этого есть формула: $$\hat{c}^2 = \cfrac{S^2(\hat{c})}{n - m}, S^2(\hat{c}) = (X\hat{c} - Y)^T(X\hat{c} - Y)$$

In [138]:
S = X @ C - Y
Ssq = S.T @ S
var = Ssq / (n - 3)
print(var)

0.11973197812338982


### Доверительные интервалы
Мы знаем, что $$\cfrac{\hat{c_i} - c_i}{\sqrt{A^{-1}_{ii}}S^2(\hat{c})} \sim T(n - m)$$ 
выражаем коэффициенты $$\hat{c_i} + T_{\alpha / 2}\sqrt{A^{-1}_{ii}}S^2(\hat{c}) < c_i < \hat{c_i} + T_{ 1 - \alpha / 2}\sqrt{A^{-1}_{ii}}S^2(\hat{c})$$

In [139]:
leftb = []
rightb = []

def count_borders(i):
    leftb.append(C[i] + t.ppf(1 - alpha / 2, n - 3) * np.sqrt(invA[i][i]) * Ssq)
    rightb.append(C[i] - t.ppf(1 - alpha / 2, n - 3) * np.sqrt(invA[i][i]) * Ssq)
    
for i in range(3):
    count_borders(i)
    
print(leftb)
print(rightb)

[0.09377809838737067, 0.0033245697964077877, 92.10941145347428]
[-0.09272544575603636, -0.0033080420073405997, -85.41426844964556]


Теперь доверительный интервал остаточной дисперсии $$\cfrac{S^2(\hat{c})}{\sigma^2} \sim \сhi^2(n - m)$$
Выражаем коэффициенты $$\cfrac{S^2(\hat{c})}{\сhi^2_{1 - \alpha / 2}(n - m)} < \sigma^2 < \cfrac{S^2(\hat{c})}{\сhi^2_{\alpha / 2}(n - m)}$$

### Коэффициент детерминации
Величина $$R = \cfrac{\sum\limits^n_{i = 1}(Y_i - \overline{Y})(\hat{Y_i} - \overline{\hat{Y}})}{\sqrt{\sum\limits^n_{i = 1}(Y_i - \overline{Y})^2\sum\limits^n_{i = 1}(\hat{Y_i} - \overline{\hat{Y}})^2}}$$
называется коэффициент детерминации

In [140]:
numerator = 0
Yhat = X @ C
for i in range(n):
    numerator += (Y[i] - np.mean(Y)) * (Yhat[i] - np.mean(Yhat))

denom1 = 0
for i in range(n):
    denom1 += (Y[i] - np.mean(Y)) ** 2
    
denom2 = 0
for i in range(n):
    denom2 += (Yhat[i] - np.mean(Yhat)) ** 2

R = numerator / (np.sqrt(denom1 * denom2))
print("Кщэффициент детерминации:", R)

Кщэффициент детерминации: 0.5183897365647344


### Проверка гипотез
Из лекций мы знаем, что: $$\cfrac{\hat{c_i} - c_i}{\sqrt{A_{ii}^{-1}} S^2(\hat{c})} \sim T(n - m)$$

#### Чем больше продажи - тем больше рейтинг
$$H_0: c_1 = 0$$ $$H_1: c_1 > 0$$
Используем T-тест - проверяем, что $T_{1 - \alpha} < \cfrac{\hat{c_1}}{\sqrt{A_{ii}^{-1}} S^2(\hat{c})}$
Если квантиль меньше - отвергаем $H_0$, иначе - нет

In [141]:
test_value = t.ppf(1 - alpha, n - 3)
check_value = C[1] / (np.sqrt(invA[1][1]) * Ssq)
print("H0 гипотеза отвергнута:", test_value < check_value)

H0 гипотеза отвергнута: False


#### Рейтинг зависит от цены
$$H_0: c_0 = 0$$ $$H_1: c_0 \neq 0$$
Используем T-тест - проверяем, что $T_{\cfrac{\alpha}{2}} < \cfrac{\hat{c_i}}{\sqrt{A_{ii}^{-1}} S^2(\hat{c})} < T_{1 - \cfrac{\alpha}{2}}$

In [142]:
left_test_value = t.ppf(alpha / 2, n - 3)
right_test_value = t.ppf(1 - alpha / 2, n - 3)
check_value = C[0] / (np.sqrt(invA[0][0]) * Ssq)
print("H0 гипотеза отвергнута:", not (left_test_value < check_value < right_test_value))

H0 гипотеза отвергнута: False


#### Равенство $c_0$ и $c_1$ нулю
Из лекций знаем, что $$\cfrac{n - m}{k} \cfrac{S^2(\hat{c_{T, t_0}}) - S^2(\hat{c})}{S^2(\hat{c})} \sim F(k, n - m)$$

Проверим, что $c_0 = c_1 = 0$ с помощью F-критерия. Возьмём T - единичную, с дополнительным столбцом из нулей, чтобы избавиться от свободного коэффициента. Таким образом, если величина выше больше $F_{1 - \alpha}$ - тогда мы отвергаем $H_0$. Кстати, вот гипотезы: $$H_0: TC = 0$$ $$H_1: TC \neq 0$$

Вычислим $F$: для числителя есть аналитическая формула $(T\hat{c} - t_0)^TD^{-1}(T\hat{c} - t_0)$, где $D = TA^{-1}T^T$

In [143]:
T = np.array([[1, 0, 0], [0, 1, 0]])
t0 = np.array([0, 0])
D = T @ invA @ T.T
invD = np.linalg.inv(D)
numerator = (T @ C - t0).T @ invD @ (T @ C - t0)
F = ((n - 2) / 2) * (numerator / Ssq)

distribution = f(2, n - 3)
test_value = distribution.ppf(1 - alpha)
print("H0 гипотеза отвергнута:", test_value < F)

H0 гипотеза отвергнута: True


### Задача 2

Проверить гипотезу о равенстве средних на каждом уровне фактора с помощью модели однофакторного дисперсионного анализа (реализовать необходимо самим)

Фактор - курит/не курит ("smoker"), выходная переменная - ИМТ ("bmi")

In [144]:
import pandas as pd
from statsmodels.formula.api import ols

df = pd.read_csv("sex_bmi_smokers.csv")
split_val = ols("bmi ~ smoker", data=df).fit()

### Гипотеза и проверка библиотечным методом:

$H_0$ - средний ИМТ равен у тех кто курит и у тех кто не курит
$H_1$ - средний ИМТ различается у курящих и не курящих


In [145]:
import statsmodels.api as sm

sm.stats.anova_lm(split_val, typ=2)

,sum_sq,df,F,PR(>F)
smoker,0.699349,1.0,0.018792,0.890985
Residual,49719.501037,1336.0,NaN,NaN


$p_value >> 0.05$, а значит можно с уверенностью сказать что $H_0$ оказалась правдивой

### Своя реализация

In [146]:
from scipy.special import betainc
import pandas as pd

def f_cdf(x, a, b):
    return betainc(a / 2, b / 2, a * x / (a * x + b))

def anova_lm(data, dependent_variable, factor_variable):
    # Разбиение данных на уровни фактора
    levels = data[factor_variable].unique()
    groups = [data[dependent_variable][data[factor_variable] == level] for level in levels]

    # Общее среднее значение
    grand_mean = data[dependent_variable].mean()

    # Вычисление сумм квадратов между группами
    ssb = sum([len(group) * (group.mean() - grand_mean)**2 for group in groups])

    # Вычисление сумм квадратов внутри групп
    ssw = sum([(value - group.mean())**2 for group in groups for value in group])

    # Вычисление общей суммы квадратов
    sst = sum((data[dependent_variable] - grand_mean)**2)

    # Вычисление степеней свободы
    dfb = len(levels) - 1
    dfw = len(data) - len(levels)
    dft = len(data) - 1

    # Вычисление средних квадратов
    msb = ssb / dfb
    msw = ssw / dfw

    # Вычисление F-статистики
    f_statistic = msb / msw

    # Вычисление p-значения
    p_value = 1 - f_cdf(f_statistic, dfb, dfw)

    return {
        'Source of Variation': ['Between Groups', 'Within Groups', 'Total'],
        'Sum of Squares': [ssb, ssw, sst],
        'Degrees of Freedom': [dfb, dfw, dft],
        'Mean Square': [msb, msw, ''],
        'F-statistic': [f_statistic, '', ''],
        'P-value': [p_value, '', '']
    }

# Применение функции anova_lm
result = anova_lm(df, dependent_variable='bmi', factor_variable='smoker')

# Вывод результатов
result_df = pd.DataFrame(result)
print(result_df)

  Source of Variation  Sum of Squares  Degrees of Freedom Mean Square  \
0      Between Groups        0.699349                   1    0.699349   
1       Within Groups    49719.501037                1336   37.215195   
2               Total    49720.200386                1337               

  F-statistic   P-value  
0    0.018792  0.890985  
1                        
2                        


Таким образом, собственноручно написанный нами метод дал результат $p_value$ такого же порядка и соответственно мы можем с уверенностью сказать что ИМТ не зависит от того курит человек или нет